We are going to build a simple decision tree model, and then build a simple random forest model, and then we will compare the accuracy and the time to build.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from scipy.stats import ttest_ind
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings("ignore")
sns.set(color_codes=True)
%matplotlib inline
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

Let's convert the data to numeric and create our input and output variables.

In [2]:
df = pd.read_csv('student-mat.csv', delimiter=';')
df['school'] = pd.Series(np.where(df.school.values == 'GP', 1, 0),
          df.index)
df['sex'] = pd.Series(np.where(df.sex.values == 'M', 1, 0),
          df.index)
df['address'] = pd.Series(np.where(df.address.values == 'U', 1, 0),
          df.index)
df['famsize'] = pd.Series(np.where(df.famsize.values == 'GT3', 1, 0),
          df.index)
df['Pstatus'] = pd.Series(np.where(df.Pstatus.values == 'A', 1, 0),
          df.index)
df['Mjob'] = pd.Series(np.where(df.Mjob.values == 'U', 1, 0),
          df.index)
df['guardian'] = pd.Series(np.where(df.guardian.values == 'mother', 1, 0),
          df.index)
df.replace('yes', 1,inplace=True)
df.replace('no', 0,inplace=True)
df.replace('teacher', 0,inplace=True)
df.replace('other', 1,inplace=True)
df.replace('services', 2,inplace=True)
df.replace('health', 3,inplace=True)
df.replace('at_home', 4,inplace=True)
df.replace('course', 0,inplace=True)
df.replace('home', 4,inplace=True)
df.replace('reputation', 1,inplace=True)
X = df.drop(['G3'], 1)
X = pd.get_dummies(X)
y = df['G3']
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,1,0,18,1,1,1,4,4,0,0,0,1,2,2,0,1,0,0,0,1,1,0,0,4,3,4,1,1,3,6,5,6,6
1,1,0,17,1,1,0,1,1,0,1,0,0,1,2,0,0,1,0,0,0,1,1,0,5,3,3,1,1,3,4,5,5,6
2,1,0,15,1,0,0,1,1,0,1,1,1,1,2,3,1,0,1,0,1,1,1,0,4,3,2,2,3,3,10,7,8,10
3,1,0,15,1,1,0,4,2,0,2,4,1,1,3,0,0,1,1,1,1,1,1,1,3,2,2,1,1,5,2,15,14,15
4,1,0,16,1,1,0,3,3,0,1,4,0,1,2,0,0,1,1,0,1,1,0,0,4,3,2,1,2,5,4,6,10,10


Let's start by building our decision tree.

In [3]:
import timeit
start_time = timeit.default_timer()

# This is the model we'll be using.
from sklearn import tree

# A convenience for displaying visualizations.
from IPython.display import Image

# Packages for rendering our tree.
import pydotplus
import graphviz

# Initialize and train our tree.
decision_tree = tree.DecisionTreeClassifier(
    criterion='entropy',
    max_features=1,
    max_depth=4,
)
decision_tree.fit(X, y)

# Render our tree.
dot_data = tree.export_graphviz(
    decision_tree, out_file=None,
    feature_names=X.columns,
    filled=True
)
graph = pydotplus.graph_from_dot_data(dot_data)
Image(graph.create_png())
print("Cross validation score:", cross_val_score(decision_tree, X, y, cv=10).mean())

elapsed = timeit.default_timer() - start_time
print("Time:", elapsed)

Cross validation score: 0.15586640327650292
Time: 1.1594306870028959


Our decision tree model is lightning quick boasting a speed time of: 1.08, though the accuracy isn't very high. Now lets build our random forest.

In [4]:
start_time = timeit.default_timer()


rfc = ensemble.RandomForestClassifier(n_jobs=-1)

rfc_fit = rfc.fit(X, y)
print("Cross validation score:", cross_val_score(rfc, X, y, cv=10).mean())

elapsed = timeit.default_timer() - start_time
print("Time:", elapsed)

Cross validation score: 0.337229110697353
Time: 2.3426658430034877


Our random forest is over two times slower than just a single decision tree, but it is also 2 times more accurate.